In [1]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

cmd = cmdgen.CommandGenerator()

In [2]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.3.6.1.2.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

errorIndication: None
errorStatus: 0
errorIndex: 0
varBinds: [ObjectType(ObjectIdentity(ObjectName('1.3.6.1.2.1.1.1.0')), DisplayString(b'Linux li744-109 4.1.5-x86_64-linode61 #7 SMP Mon Aug 24 13:46:31 EDT 2015 x86_64', subtypeSpec=ConstraintsIntersection(ConstraintsIntersection(ConstraintsIntersection(ConstraintsIntersection(), ValueSizeConstraint(0, 65535)), ValueSizeConstraint(0, 255)), ValueSizeConstraint(0, 255))))]


chexiang
===

- system = 1.3.6.1.2.1.1
- sysName = 1.3.6.1.2.1.1.5
- ifEntry = 1.3.6.1.2.1.2.2.1
- ifDescr = 1.3.6.1.2.1.2.2.1.2
- ifSpeed = 1.3.6.1.2.1.2.2.1.5

## 同步

In [10]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
cmdGen = cmdgen.CommandGenerator()  # 同步

mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
)
mibView = view.MibViewController(mibBuilder)



errorIndication, errorStatus, errorIndex, varBinds = cmdGen.nextCmd(
    cmdgen.CommunityData('huawei@123'),
    cmdgen.UdpTransportTarget(('10.32.33.16', 161)),
    cmdgen.MibVariable('IF-MIB', 'ifOperStatus')
)


print(errorIndication)
print(errorStatus)
print(errorIndex)
print('*' * 50)

for row in varBinds:
    for oid, val in row:
        print(oid.getMibSymbol()[-1][0], val.prettyPrint())

None
0
0
**************************************************
1 'up'
2 'up'
3 'down'
5 'up'
6 'up'
7 'up'
8 'up'
9 'up'
10 'up'
11 'up'
12 'up'
13 'down'
14 'down'
15 'up'
16 'up'
17 'up'
18 'up'
19 'up'
20 'up'
21 'up'
22 'up'
23 'down'
24 'down'
25 'up'
26 'up'
27 'up'
28 'up'
29 'up'
30 'up'
31 'up'
32 'up'
33 'up'
34 'up'
35 'up'
36 'up'
37 'up'
38 'up'
39 'up'
40 'up'
41 'up'
42 'up'
43 'up'
44 'up'
45 'up'
46 'up'
47 'up'
48 'up'
49 'down'
50 'down'
51 'down'
52 'down'
53 'up'
54 'down'
55 'up'
56 'up'
57 'up'
58 'up'
60 'up'
61 'up'
62 'up'
63 'up'
64 'up'
65 'up'
66 'up'
67 'up'
68 'up'
69 'down'
70 'up'
71 'up'
72 'up'
73 'up'
74 'up'
75 'up'
76 'up'
77 'up'
78 'down'
79 'down'
80 'up'
81 'up'
82 'up'
83 'up'
84 'up'
85 'up'
86 'up'
87 'up'
88 'up'
89 'up'
90 'up'
91 'up'
92 'up'
93 'up'
94 'up'
95 'up'
96 'up'
97 'up'
98 'up'
99 'up'
100 'up'
101 'up'
102 'up'
103 'up'
104 'down'
105 'down'
106 'down'
107 'down'
108 'up'
109 'down'
110 'up'
111 'up'
112 'up'
113 'up'
114 'up'
1

## 异步

In [12]:
import string
# from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
# cmdGen = cmdgen.CommandGenerator()  # 同步
cmdGen = cmdgen.AsynCommandGenerator()  # 异步

# mibBuilder = builder.MibBuilder()
# mibBuilder.loadModules(
#     'RFC-1213', 'RFC-3413'
# )
# mibView = view.MibViewController(mibBuilder)


mib_vars = (
    {'desc': 'ifOperStatus', 'mib': cmdgen.MibVariable('IF-MIB', 'ifOperStatus')},  # 端口状态
#     {'desc': 'ifSpeed', 'mib': cmdgen.MibVariable('IF-MIB', 'ifSpeed')},  # 端口速度 bits/s

)

ttt = []
def callback(sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
    
    for row in varBinds:
        for oid, val in row:
            if cbCtx['object_name'] not in oid.prettyPrint():
                return 0
            
            print(oid.prettyPrint(), val.prettyPrint())
            ttt.append(oid)
#             return
            
    return 1  # 告诉 walk 继续运行
            


for mib_var in mib_vars:
    cbCtx = {'object_name': mib_var['desc']}
    cmdGen.nextCmd(
        cmdgen.CommunityData('huawei@123'),
        cmdgen.UdpTransportTarget(('10.32.33.16', 161)),
        (mib_var['mib'],),
        (callback, cbCtx),
        True, True
    )

    
# 启动
cmdGen.snmpEngine.transportDispatcher.jobStarted(1)
cmdGen.snmpEngine.transportDispatcher.runDispatcher()
cmdGen.snmpEngine.transportDispatcher.closeDispatcher()

IF-MIB::ifOperStatus."1" 'up'
IF-MIB::ifOperStatus."2" 'up'
IF-MIB::ifOperStatus."3" 'down'
IF-MIB::ifOperStatus."5" 'up'
IF-MIB::ifOperStatus."6" 'up'
IF-MIB::ifOperStatus."7" 'up'
IF-MIB::ifOperStatus."8" 'up'
IF-MIB::ifOperStatus."9" 'up'
IF-MIB::ifOperStatus."10" 'up'
IF-MIB::ifOperStatus."11" 'up'
IF-MIB::ifOperStatus."12" 'up'
IF-MIB::ifOperStatus."13" 'down'
IF-MIB::ifOperStatus."14" 'down'
IF-MIB::ifOperStatus."15" 'up'
IF-MIB::ifOperStatus."16" 'up'
IF-MIB::ifOperStatus."17" 'up'
IF-MIB::ifOperStatus."18" 'up'
IF-MIB::ifOperStatus."19" 'up'
IF-MIB::ifOperStatus."20" 'up'
IF-MIB::ifOperStatus."21" 'up'
IF-MIB::ifOperStatus."22" 'up'
IF-MIB::ifOperStatus."23" 'down'
IF-MIB::ifOperStatus."24" 'down'
IF-MIB::ifOperStatus."25" 'up'
IF-MIB::ifOperStatus."26" 'up'
IF-MIB::ifOperStatus."27" 'up'
IF-MIB::ifOperStatus."28" 'up'
IF-MIB::ifOperStatus."29" 'up'
IF-MIB::ifOperStatus."30" 'up'
IF-MIB::ifOperStatus."31" 'up'
IF-MIB::ifOperStatus."32" 'up'
IF-MIB::ifOperStatus."33" 'up'
IF-MIB

KeyboardInterrupt: 